In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# set a rsandom seed for reproducibility
torch.manual_seed(42)

In [3]:
df = pd.read_csv('fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [4]:
# train test split

x = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [5]:
X_train , X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [6]:
#scaling the features

X_train = X_train/255.0
X_test = X_test/255.0

In [7]:
# Create a CustomDataset class
class CustomDataset(Dataset):
    def __init__(self, Features, Labels):
        self.Features = torch.tensor(Features, dtype=torch.float32)
        self.Labels = torch.tensor(Labels, dtype=torch.long)
        
    def __len__(self):
        return len(self.Features)
    
    def __getitem__(self, idx):
        return self.Features[idx], self.Labels[idx]


In [8]:
# create a train_dataset object 
train_dataset = CustomDataset(X_train, y_train)

In [9]:
len(train_dataset)

4800